# 抓取 Googleplay 的 APP 評論與評分

## 安裝套件

In [0]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubu

## 載入套件

In [0]:
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd 
import time

## 打開想爬的網頁
在此以 Foodpanda 的APP頁面作為範例

https://play.google.com/store/apps/details?id=com.global.foodpanda.android&hl=zh_TW&showAllReviews=true


In [0]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-notifications")
driver = webdriver.Chrome(options=options)
driver.get('https://play.google.com/store/apps/details?id=com.global.foodpanda.android&hl=zh_TW&showAllReviews=true')
time.sleep(1)
driver.find_element_by_xpath('//div[@class="MocG8c UFSXYb LMgvRb KKjvXb"]').click()
time.sleep(1)
try:
    driver.find_element_by_xpath('//div[@jsname="V68bde"]//span[@class="vRMGwf oJeWuf"]').click()
except:
    pass

## 加載更多留言
- 透過觀察的方式可以發現，當網頁滾動到最底部的時候會自動加載更多留言，但是每加載5次會需要點擊一次「顯示更多內容」
- 另外需要留意第一次與第二次之後的「顯示更多內容」是不同的按鈕，所以在這裡使用了一個 try 函數
- 在這裡我設定做 10個點擊「顯示更多內容」，實際上可以依據資料量的大小自行增加或減少

In [0]:
for i in range(10):
    for i in range(5):
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        driver.find_element_by_xpath('//div[@class="U26fgb O0WRkf oG5Srb C0oVfc n9lfJ M9Bg4d"]').click()
    except:
        try:
            driver.find_element_by_xpath('//div[@class="U26fgb O0WRkf oG5Srb C0oVfc n9lfJ"]').click()
        except:
            pass

## 檢視完整留言
- 對於長度較長的留言需要另外點擊「完整留言」的按鈕才會顯示完整留言，顯示後我們才能抓到資料!
- 這裡我找的是頁面中有沒有「完整留言」的按鈕可以點擊，有的話就點擊

In [0]:
SeeMore = '//button[@jsname="gxjVle"]'
driver.execute_script("window.scrollTo(0, 0);")
for i, element in enumerate(driver.find_elements_by_xpath(SeeMore)):
    if i % 10 == 0:
        print(i)
    element.click()
    print(n)

## 解析資料
- 當我們把所有資料加載完成後，我們的資料是html的半結構化的網頁資料
- 因此我們在這裡需要透過BeautifulSoup來幫我們把資料轉成DataFrame

In [0]:
soup = BeautifulSoup(driver.page_source)
driver.close()

In [0]:
list_of_comments = []
for i, element in enumerate(soup.find_all('div',{'jscontroller':'H6eOGe'})):
    if i % 10 == 0:
        print(i)
    try:
        likecounts = element.find('div',{'class':'jUL89d y92BAb'}).text
    except:
        likecounts = str(0)
    
    Content = element.find('span',{'jsname':'bN97Pc'}).text
    # 如果「完整評論」裡面有內容就抓裡面的內容
    if len(element.find('span',{'jsname':'fbQN7e'}).text) >=20:
        Content = element.find('span',{'jsname':'fbQN7e'}).text
    ndf = pd.DataFrame([{'Name':element.find('span', {'class':'X43Kjb'}).text,
                         'Sentimen':element.find('div', {'class':'pf5lIe'}).find('div')['aria-label'],
                         'likecounts':likecounts,
                         'Timestamp':element.find('span',{'class':'p2TkOb'}).text,
                         'Content':Content}],
                        columns = ['Name', 'Timestamp', 'Content', 'Sentimen', 'likecounts'])
    list_of_comments.append(ndf)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


In [0]:
comments = pd.concat(list_of_comments, ignore_index=True)
print(len(comments))
comments.head(20)

200


,Name,Timestamp,Content,Sentimen,likecounts
0,蔡雁斗,2019年12月26日,好用,獲評為 5 顆星 (滿分 5 顆星),0
1,蘇勇銓,2019年12月26日,很好用,獲評為 5 顆星 (滿分 5 顆星),0
2,鄭家柔,2019年12月26日,很好,獲評為 5 顆星 (滿分 5 顆星),0
3,李梓柔,2019年12月26日,實用,獲評為 5 顆星 (滿分 5 顆星),0
4,Jill Lu,2019年12月26日,很方便,獲評為 5 顆星 (滿分 5 顆星),0
5,林佩瑾,2019年12月26日,非常好用的app,獲評為 5 顆星 (滿分 5 顆星),0
6,tim chen,2019年12月26日,垃圾系統,獲評為 1 顆星 (滿分 5 顆星),0
7,Shen Le,2019年12月26日,按結帳完全沒反應 那幹嘛還要用這個app 換另一個還比你們好用,獲評為 1 顆星 (滿分 5 顆星),0
8,Lango Chen,2019年12月26日,方便,獲評為 5 顆星 (滿分 5 顆星),0
9,吉光片羽,2019年12月26日,好,獲評為 5 顆星 (滿分 5 顆星),0


## 保存資料
- 先跟 Google Drive 空間串接，再將資料存在 Google Drive 上
- 資料放在 Google Drive 上的 Colab Notebooks 資料夾中

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
path = '/content/drive/My Drive/Colab Notebooks/'
comments.to_excel(path + 'GooglePlay_Comments.xlsx')